<a href="https://colab.research.google.com/github/ashesh-12/Advanced-Chatbot/blob/main/Chapter2_Demo1_Flan_T5_Chatbot_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 599.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 118.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 114.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 131.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.9 MB/s eta 0:00:00


In [15]:
!pip uninstall -y keras
!pip install tf-keras

Found existing installation: keras 3.8.0
Uninstalling keras-3.8.0:
  Successfully uninstalled keras-3.8.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.6 MB/s eta 0:00:00


In [2]:
!sudo apt-get remove python3-blinker

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages were automatically installed and are no longer required:
  distro-info-data gir1.2-packagekitglib-1.0 libappstream4
  libpackagekit-glib2-18 libpolkit-agent-1-0 libpolkit-gobject-1-0
  libstemmer0d libxmlb2 libyaml-0-2 lsb-release packagekit pkexec policykit-1
  polkitd python-apt-common python3-apt python3-cffi-backend
  python3-cryptography python3-dbus python3-distro python3-gi python3-httplib2
  python3-jeepney python3-jwt python3-keyring python3-lazr.uri
  python3-pkg-resources python3-pyparsing python3-secretstorage python3-six
  python3-wadllib
Use 'sudo apt autoremove' to remove them.
The following packages will be REMOVED:
  python3-blinker python3-launchpadlib python3-lazr.restfulclient
  python3-oauthlib python3-software-properties software-properties-common
0 upgraded, 0 newly installed, 6 to remove and 0 not upgraded.
After this operation, 3,022 kB disk s

In [3]:
!pip install flask blinker

  Using cached flask-3.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
Using cached flask-3.1.0-py3-none-any.whl (102 kB)
Using cached blinker-1.9.0-py3-none-any.whl (8.5 kB)


In [4]:
!pip install transformers==4.41.2 # Replace with an appropriate version

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 97.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3


In [16]:
%%writefile flask_chatbot_app.py
from flask import Flask, request, jsonify
from transformers import pipeline, TFAutoModelForSeq2SeqLM, AutoTokenizer

app = Flask(__name__)

# Load the Hugging Face model

model_name = 'google/flan-t5-xl'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)
sentiment_model = pipeline("sentiment-analysis")

@app.route('/chatbot', methods=['POST'])
def chatbot():
    user_input = request.json.get('message')
    if not user_input:
        return jsonify({'response': 'Please provide a message.'}), 400
    sentiment = sentiment_model(user_input)[0]
    response = generate_response(user_input, sentiment)
    return jsonify({'response': response})

def generate_response(user_input, sentiment):
    if sentiment['label'] == 'NEGATIVE':
        prompt = f"The user is upset, respond with empathy and support: {user_input}"
    else:
        prompt = f"Respond to the following query: {user_input}"
    input_ids = tokenizer.encode(prompt, truncation=True, padding=True, max_length=512, return_tensors="tf")
    output = model.generate(input_ids, max_length=250, num_beams=5, early_stopping=True)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response


if __name__ == '__main__':
    app.run(debug=True, port=5000, host='0.0.0.0')


Overwriting flask_chatbot_app.py


In [17]:

import subprocess

# Stop any running Flask app
subprocess.run(['pkill', '-f', 'flask_chatbot_app.py'])

CompletedProcess(args=['pkill', '-f', 'flask_chatbot_app.py'], returncode=1)

In [18]:
!nohup python flask_chatbot_app.py &

nohup: appending output to 'nohup.out'


In [27]:
!sudo lsof -i -P -n | grep LISTEN

node          7 root   21u  IPv6  15453      0t0  TCP *:8080 (LISTEN)
kernel_ma    36 root    3u  IPv4  30822      0t0  TCP 172.28.0.12:6000 (LISTEN)
colab-fil   202 root    3u  IPv4  21817      0t0  TCP 127.0.0.1:3453 (LISTEN)
jupyter-n   231 root    7u  IPv4 106628      0t0  TCP 172.28.0.12:9000 (LISTEN)
python3    5357 root   28u  IPv4 191958      0t0  TCP 127.0.0.1:38499 (LISTEN)
python3    5443 root    3u  IPv4 168578      0t0  TCP 127.0.0.1:44963 (LISTEN)
python3    5443 root    5u  IPv4 168579      0t0  TCP 127.0.0.1:57375 (LISTEN)
python3    8980 root    9u  IPv4 291509      0t0  TCP *:5000 (LISTEN)
python3   10055 root    9u  IPv4 291509      0t0  TCP *:5000 (LISTEN)


In [30]:

import requests

# Define the URL of the Flask app
url = 'http://127.0.0.1:5000/chatbot'

# Send a request to the Flask app
response = requests.post(url, json={"message": "Explain what machine learning is."})
print(response.json())


{'response': 'Machine learning (ML) is a branch of computer science that uses algorithms to learn from data.'}


In [29]:
response = requests.post(url, json={'message': 'I was wondering how can I go to Eiffel Tower from the airport using the train and subway?'})
print(response.json())

{'response': 'The best way to get to the Eiffel Tower from the airport is by train. You can take the RER B train to the Eiffel Tower and then change to the RER C train to the Eiffel Tower.'}


In [32]:
response = requests.post(url, json={"message": "I'm sad and don't know who to talk to."})
print(response.json())

{'response': "You can talk to me. I'm here for you."}
